In [2]:
!pip install azureml-sdk


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


**Importing the dependencies**

In [3]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [4]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [6]:
print(resource_group)
print(workspace_name)
print(region)

ml-model-deployment-rg
deploy-sklearn-model-ws
centralindia


**Create a Resource Group from Azure Portal**

In [7]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-44b8aada47
Deploying StorageAccount with name deployskstorage58ab4a9af.
Deploying KeyVault with name deployskkeyvault0cccf204.
Deploying Workspace with name deploy-sklearn-model-ws.
Deploying AppInsights with name deployskinsights8b6b676b.
Deployed AppInsights with name deployskinsights8b6b676b. Took 36.98 seconds.
Deployed Workspace with name deploy-sklearn-model-ws. Took 33.46 seconds.
Workspace deploy-sklearn-model-ws created


In [8]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [9]:
model_name='diabetes_prediction_model'

In [10]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [11]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [12]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [13]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [14]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_18897/3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-01-07 13:25:51+05:30 Creating Container Registry if not exists..
2024-01-07 13:35:51+05:30 Registering the environment..
2024-01-07 13:35:55+05:30 Building image..
2024-01-07 13:46:10+05:30 Generating deployment configuration.
2024-01-07 13:46:11+05:30 Submitting deployment to compute..
2024-01-07 13:46:19+05:30 Checking the status of deployment diabetes-prediction-service..
2024-01-07 13:48:02+05:30 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
scoring_uri = service.scoring_uri

In [16]:
scoring_uri

'http://82a1dd63-7ec8-49e6-8668-c222ce56cb2e.centralindia.azurecontainer.io/score'

**Cleaning up all the created resources**

In [17]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

In [18]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


In [19]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources